In [1]:
# the aim is to end up with train, test, evaluate
import sys
sys.path.append("/Users/david/Documents/code/fhmm/v1")

In [7]:
import src.ads_davidjames9610.ads as ads
import src.ads_davidjames9610.useful as useful
import numpy as np
import os
import os.path
from pathlib import Path
import matplotlib.pyplot as plt
from importlib import reload
from src.classifiers_davidjames9610.test_a.config import *
import src.misc_davidjames9610.fe_methods as fe
import src.misc_davidjames9610.proc_methods as pm

"""
we want to end up with this

    section 2, samples to features,
    -- choose fe_method,
    -- noise and reverb, apply to test / evaluate before fe method,

    select fe_methods,
    -- this point also is important for choosing cross-validation, and train-test-eval split
    - fe_methods = ['mfcc', 'log-power', 'lfcc']
    - select noise or reverb to be applied to eval and test
    - post_proc = ['noise', 'reverb']

    -- labels don't change

    features = {
        'mfcc':
            'train': [],
            'train_labels': [cv_index_0] [cv_index_1]
            'test': [],
            'test_labels': []   // test and eval will need different process method if noise is being added
            'eval': [],
            'eval_labels': []
        'lfcc':
            ...
    }
"""

In [8]:
import pickle
sls = pickle.load(open('samples_labels.pickle','rb'))

In [9]:
import importlib
importlib.reload(fe)
nfft = 128

# processing = [
#     {'fe': fe.FeatureExtractorLogPower(nfft),
#      'noise': None,
#      'reverb': None,
#      'name': 'lp_clean'     # function ?
#      }
# ]

# proc_methods_pre = []
# proc_methods_post = []
# signal_to_noise_ratios = [None, 20, 10]

fe_methods = [fe.FeatureExtractorLogPower(nfft=128), fe.FeatureExtractorMfcc(nfft=128, num_ceps=8, fs=sr)]

In [10]:
# going to have to do fe after cross-validation split
import src.misc_davidjames9610.cross_validation as cv
from src.misc_davidjames9610.cross_validation import *
import importlib
importlib.reload(cv)

# define config things
n_folds = 2
cv_output = cv.split_data_for_cross_validation(sls['samples'], sls['labels'], n_folds, test_ratio=0.1)

In [15]:
reload(fe)

features = {
}

for fe_method in fe_methods:
    # for extra proc in proc ?
    fe_name = fe_method.__str__()
    features[fe_name] = {}

    # train
    train_features = []
    train_labels = []
    for i in range(n_folds):
        curr_features = [fe_method(sample) for sample in cv_output['train_data'][i]]
        curr_labels = cv_output['train_label'][i]
        train_features.append(curr_features)
        train_labels.append(curr_labels)

        # ng inf error handler
        # mask = [np.isneginf(array).any() for array in curr_features]
        # train_features.append([feat for feat in curr_features if not mask])
        # mask = np.invert(np.array(mask))
        # train_labels.append(cv_output['train_label'][i][mask])

    features[fe_name]['train_features'] = train_features
    features[fe_name]['train_label'] = train_labels

    # val
    val_features = []
    val_labels = []
    for i in range(n_folds):
        # no noise added her since this is used for training
        val_features.append([fe_method(sample) for sample in cv_output['val_data'][i]])
    features[fe_name]['val_features'] = val_features
    features[fe_name]['val_label'] = cv_output['val_label']

    # test
    # TODO add in noise and reverb here if required
    test_features = [fe_method(sample) for sample in cv_output['test_data']]
    features[fe_name]['test_features'] = test_features
    features[fe_name]['test_label'] = cv_output['test_label']


In [16]:
import pickle
pickle.dump(features, open('features.pickle','wb'))